In [ ]:
! pip install datasets

In [ ]:
import re
from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch

processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model.to(device)

# prepare decoder inputs
task_prompt = "<s_cord-v2>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

In [ ]:
# this part is to fetch receipt image to test
from datasets import load_dataset
dataset = load_dataset("hf-internal-testing/example-documents", split="test")

In [ ]:
# Get image
image = dataset[2]["image"]
# Display image
display(dataset[2]["image"])

In [ ]:
# # Can also pick self uploaded image to test
# import cv2
# image = cv2.imread("R1.jpg")
# #print(image)

In [ ]:
pixel_values = processor(image, return_tensors="pt").pixel_values

outputs = model.generate(
  pixel_values.to(device),
  decoder_input_ids = decoder_input_ids.to(device),
  max_length = model.decoder.config.max_position_embeddings,
  pad_token_id = processor.tokenizer.pad_token_id,
  eos_token_id = processor.tokenizer.eos_token_id,
  use_cache=True,
  bad_words_ids=[[processor.tokenizer.unk_token_id]],
  return_dict_in_generate=True,
)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
json = processor.token2json(sequence)
print(json)

## This is for Debug purpose or to download **model**

In [ ]:
# Check path of file (DEBUG)
import os
print(os.path.dirname(os.path.abspath("__file__")))

In [ ]:
# Save Model
import pickle
filename = 'image_receipt_processor_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [21]:
!zip -r image_receipt_processor_model.sav.zip image_receipt_processor_model.sav

  adding: image_receipt_processor_model.sav (deflated 7%)


In [23]:
# Check size of the model (DEBUG)
import os
gb = os.path.getsize(filename)/1024/1024
print(gb)

768.9302835464478


In [ ]:
# Download model
from google.colab import files
files.download(filename)